<a href="https://colab.research.google.com/github/Christophe-Gauge/NEOWISE/blob/master/comet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install skyfield
from skyfield.api import Loader, Topos
from skyfield.data import mpc
from skyfield.constants import GM_SUN_Pitjeva_2005_km3_s2 as GM_SUN
from skyfield import almanac

import dateutil.parser
from datetime import timedelta

import argparse

import sys, os

load = Loader('./skyfield')
with load.open(mpc.COMET_URL) as f:
    comets = mpc.load_comets_dataframe(f)

# Index by designation for fast lookup.
comets = comets.set_index('designation', drop=False)

ts = load.timescale(builtin=True)
eph = load('de421.bsp')
sun = eph['sun']
earth = eph['earth']


def comet_by_name(namepat):
    # Exact search: row = comets.loc[cometname]

    # How to search for something containing a string in pandas:
    rows = comets[comets['designation'].str.contains(namepat)]

    # Found it!
    if len(rows) == 1:
        return rows.iloc[0]

    # No match
    if len(rows) == 0:
        return None

    # Multiple matches, so print them but return nothing.
    # pandas iterrows() returns two things but they're both the same object,
    # not an index and an object like you might expect. So ignore r.
    print("Matches from", len(comets), 'comets loaded:')

    for r, row in rows.iterrows():
        print(row['designation'])
    return None


def calc_comet(comet_df, obstime, earthcoords, numdays):
    # Generating a position.
    cometvec = sun + mpc.comet_orbit(comet_df, ts, GM_SUN)

    cometpos = earth.at(obstime).observe(cometvec)
    ra, dec, distance = cometpos.radec()
    print("RA", ra, "   DEC", dec, "   Distance", distance)

    if earthcoords:
        if len(earthcoords) > 2:
            elev = earthcoords[2]
        else:
            elev = 0
        obstopos = Topos(latitude_degrees=earthcoords[0],
                         longitude_degrees=earthcoords[1],
                         elevation_m=elev)
        print("\nObserver at",
               obstopos.latitude, "N  ", obstopos.longitude, "E  ",
              "Elevation", obstopos.elevation.m, "m")
        observer = earth + obstopos

        alt, az, distance = \
            observer.at(obstime).observe(cometvec).apparent().altaz()
        print("Altitude", alt, "     Azumuth", az, distance)

        if numdays:
            print("\nRises and sets over", numdays, "days:")
            datetime1 = obstime.utc_datetime() - timedelta(hours=2)
            t0 = ts.utc(datetime1)
            t1 = ts.utc(datetime1 + timedelta(days=numdays))

            alm = almanac.risings_and_settings(eph, cometvec, obstopos)
            t, y = almanac.find_discrete(t0, t1, alm)

            fmt = "%-10s    %-10s %-8s   %-10s %-18s"
            print(fmt % ("Date", "Rise", "Azimuth", "Set", "Az"))
            datestr = ''
            risetime = ''
            riseaz = ''
            settime = ''
            setaz = ''
            for ti, yi in zip(t, y):
                alt, az, distance = \
                    observer.at(ti).observe(cometvec).apparent().altaz()
                t = ti.utc_datetime().astimezone()
                d = t.strftime("%Y-%m-%d")
                if yi:
                    risetime =  ti.utc_datetime().astimezone() \
                                                 .strftime("%H:%M %Z")
                    riseaz = "%3d°%2d'" % az.dms()[:2]
                else:
                    settime =  ti.utc_datetime().astimezone() \
                                                .strftime("%H:%M %Z")
                    setaz = "%3d°%2d'" % az.dms()[:2]
                if not datestr:
                    datestr = d
                if d != datestr:
                    print(fmt % (datestr, risetime, riseaz, settime, setaz))
                    risetime = ''
                    riseaz = ''
                    settime = ''
                    setaz = ''
                    datestr = d

            if risetime or settime:
                print(fmt % (datestr, risetime, riseaz, settime, setaz))

coords = [34.398, -118.483, 500]
comet_df = comet_by_name('2020 F3')
numdays = 5
t = ts.now()
calc_comet(comet_df, t, coords, numdays)

RA 08h 13m 18.91s    DEC +47deg 48' 51.4"    Distance 0.751796 au

Observer at 34deg 23' 52.8" N   -118deg 28' 58.8" E   Elevation 500.0 m
Altitude 46deg 40' 08.8"      Azumuth 54deg 26' 08.4" 0.751765 au

Rises and sets over 5 days:
Date          Rise       Azimuth    Set        Az                
2020-07-17    11:16 UTC   24°47'    06:20 UTC  335°23'           
2020-07-18    11:33 UTC   24°36'    06:37 UTC  335° 5'           
2020-07-19    11:53 UTC   25° 3'    06:50 UTC  334°10'           
2020-07-20    12:17 UTC   26° 6'    07:01 UTC  332°41'           
2020-07-21    12:42 UTC   27°43'                                 
